In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import math
import numpy as np

In [2]:
url = 'https://stackoverflow.com/jobs'

In [3]:
response = requests.get(url)
soup = BeautifulSoup(response.content, features = 'lxml')

## Find nb of pages

In [4]:
jobs_nb = soup.find('div', attrs = {'class':'js-search-title'}).findChild().text.replace(',','')
jobs_nb = re.findall('\d+', jobs_nb)[0]

pages = math.ceil((int(jobs_nb)/25))
jobs_nb


'16505'

Finding the number of pages is not workin for StackOverflow, I don't really get why

## Set url to scrape list of jobs

In [5]:
pages = 150
urls = ['https://stackoverflow.com/jobs?pg='+ str(page) for page in range(1,pages+1)]

In [6]:
responses =[requests.get(url) for url in urls]

In [7]:
soups = [BeautifulSoup(response.content,'lxml') for response in responses]

In [8]:
a_tags = [soup.find_all('a', {'class':'s-link stretched-link'}) for soup in soups]
titles = [a.text for sublist in a_tags for a in sublist]


In [9]:
links = ['https://stackoverflow.com' + a['href'] for sublist in a_tags for a in sublist]

## Tech-tags given by website

In [10]:
# tech_tags = [soup.find_all('a', attrs ={'class': 'post-tag'})]
# technologies = [a.text for tag in tech_tags for a in tag]
# list(set(technologies))

## Set url for scraping the job descriptions

In [11]:
description_responses =[requests.get(url) for url in links]

In [12]:
description_soups = [BeautifulSoup(response.content,'lxml') for response in description_responses]

**Tech Tags**:

In [13]:
tags = [soup.find_all('a', attrs = {'class':'post-tag'}) for soup in description_soups]


**Job details**:

In [14]:
job_details = [soup.find_all('div', attrs = {'class':'job-details--about'}) for soup in description_soups]

job_det = []

for job in job_details:
    if len(job):
        if len(job[0].find_all('span')) > 2:
            job_det.append([job[0].find_all('span')[3].text.lower()])
        else:
            job_det.append(['other'])  
    else:
        job_det.append(['other'])

job_det

[['mid-level, senior'],
 ['mid-level, senior'],
 ['senior, lead'],
 ['full stack developer'],
 ['consumer electronics'],
 ['mid-level, senior, lead'],
 ['other'],
 ['information technology, internet of things, marketing'],
 ['frontend developer'],
 ['product manager'],
 ['other'],
 ['senior'],
 ['mid-level, senior'],
 ['mid-level'],
 ['mid-level, senior'],
 ['backend developer'],
 ['mid-level, senior'],
 ['other'],
 ['other'],
 ['other'],
 ['other'],
 ['other'],
 ['mid-level, senior'],
 ['mid-level, senior, lead'],
 ['frontend developer'],
 ['senior'],
 ['junior, mid-level, senior'],
 ['senior, lead'],
 ['other'],
 ['junior, mid-level, senior'],
 ['senior'],
 ['mid-level'],
 ['data scientist'],
 ['other'],
 ['other'],
 ['system administrator'],
 ['mid-level, senior'],
 ['junior, mid-level, senior'],
 ['backend developer'],
 ['lead, manager'],
 ['mid-level, senior'],
 ['junior, mid-level, senior'],
 ['junior, mid-level, senior'],
 ['other'],
 ['other'],
 ['mid-level, senior'],
 ['junior

## Find technologies in golden source

In [15]:
technologies = pd.read_json('golden-source-classified.json')

pattern_languages = '|'.join(technologies['remote_project']['technologies']['languages'])
pattern_web_frameworks = '|'.join(technologies['remote_project']['technologies']['web_frameworks'])
pattern_libraries = '|'.join(technologies['remote_project']['technologies']['other_frameworks_libraries_tools'])
pattern_databases = '|'.join(technologies['remote_project']['technologies']['databases'])
pattern_platforms = '|'.join(technologies['remote_project']['technologies']['platforms'])
pattern_other = '|'.join(technologies['remote_project']['technologies']['other_habilities'])

pattern_all_technologies = pattern_languages + '|' + pattern_web_frameworks + '|' + pattern_libraries + '|' + pattern_databases + '|' +pattern_platforms + '|' +pattern_other

In [16]:
languages = [list(set(re.findall(pattern_languages,str(tag).replace("-"," ").lower()))) for tag in tags]
web_frameworks = [list(set(re.findall(pattern_web_frameworks,str(tag).replace("-"," ").lower()))) for tag in tags]
libraries = [list(set(re.findall(pattern_libraries,str(tag).replace("-"," ").lower()))) for tag in tags]
databases = [list(set(re.findall(pattern_databases,str(tag).replace("-"," ").lower()))) for tag in tags]
platforms = [list(set(re.findall(pattern_platforms,str(tag).replace("-"," ").lower()))) for tag in tags]
other = [list(set(re.findall(pattern_other,str(tag).replace("-"," ").lower()))) for tag in tags]

## Find level of experience

In [17]:
level = pd.read_json('golden-source-classified.json')
pattern = r'\b|\b'.join(level['remote_project']['level'])

pattern

entry_levels = [(set(re.findall(pattern, str(job).lower()))) for job in job_det]


entry_levels_clean = []

for level in entry_levels:
    if level.intersection({'senior','experienced','sr','sr.','manager'}):
        entry_levels_clean.append('senior')
    elif level.intersection({'junior','jr','jr.'}):
        entry_levels_clean.append('junior')
    else:
        entry_levels_clean.append('other')
    


## DataFrame

In [18]:
df = pd.DataFrame(titles,columns = ["Title"])
df['Link'] = links

df["Languages"] = [",".join(techs) for techs in languages]
df["Web Frameworks"] = [",".join(techs) for techs in web_frameworks]
df["Libraries"] = [",".join(techs) for techs in libraries]
df["Databases"] = [",".join(techs) for techs in databases]
df["Platforms"] = [",".join(techs) for techs in platforms]
df["Other"] = [",".join(techs) for techs in other]

df['Experience Level'] = [level for level in entry_levels_clean]

df

,Title,Link,Languages,Web Frameworks,Libraries,Databases,Platforms,Other,Experience Level
0,Software Engineers (m/f/d) for .NET technologi...,https://stackoverflow.com/jobs/368878/software...,"javascript,c,java","angular,spring",,,,,senior
1,Senior Software Engineer - Cloud,https://stackoverflow.com/jobs/376878/senior-s...,"asp,html,c",,"git,.net",,,,senior
2,Lead/Senior Backend Developer - We're motivate...,https://stackoverflow.com/jobs/268694/lead-sen...,"javascript,java",spring,,,kubernetes,,senior
3,"Senior Software Engineer, Full-Stack",https://stackoverflow.com/jobs/354953/senior-s...,"javascript,c,java,python","angular,react",,"elasticsearch,postgresql",,,other
4,"App Store Engineering, Software Engineer",https://stackoverflow.com/jobs/354910/app-stor...,"javascript,c,java,python","angular,react",,"elasticsearch,postgresql",,,other
...,...,...,...,...,...,...,...,...,...
3745,PHP Developer,https://stackoverflow.com/jobs/375194/php-deve...,"javascript,php",,,mysql,,,other
3746,PHP Developer,https://stackoverflow.com/jobs/375146/php-deve...,"javascript,php",,,mysql,,,other
3747,Php Developer,https://stackoverflow.com/jobs/329568/php-deve...,"javascript,php",,,mysql,,,other
3748,PHP Developer,https://stackoverflow.com/jobs/375141/php-deve...,"javascript,php",,,mysql,,,other


## Save DataFrame


In [19]:
df.to_csv('dataframes/non-remote-stackoverflow.csv', index = False)

## Create a dictionary to count the times a technology is required

In [20]:
def create_tech_dict(col_name,df):
    technologies_set = df[col_name]
    technologies_dict= {}
    for technologies in technologies_set:
        for technology in technologies.split(','):
            if technology in technologies_dict.keys():
                technologies_dict[technology] += 1
            else:
                technologies_dict[technology] =  1
    return {key: value for key, value in sorted(technologies_dict.items(), key=lambda item: item[1])}

In [21]:
create_tech_dict('Libraries',df)
create_tech_dict('Web Frameworks',df)
create_tech_dict('Languages',df)
# create_tech_dict('Databases',df)
# create_tech_dict('Platforms',df)
# create_tech_dict('Other',df)

{'webassembly': 1,
 'assembly': 1,
 'vba': 2,
 'f#': 2,
 'dart': 4,
 'matlab': 6,
 'golang': 10,
 'shell': 12,
 'rust': 19,
 'elixir': 21,
 'powershell': 29,
 'clojure': 37,
 'r': 40,
 'bash': 43,
 'sass': 59,
 'graphql': 89,
 'swift': 139,
 'objective c': 144,
 'scala': 162,
 '': 191,
 'kotlin': 252,
 'ruby': 263,
 'go': 271,
 'asp': 280,
 'typescript': 371,
 'html': 374,
 'css': 413,
 'sql': 610,
 'php': 739,
 'c': 859,
 'python': 1198,
 'java': 1341,
 'javascript': 1940}